In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
%cd S:\PPMI_RNA\Processed\Progression_Analysis\Male
data = pd.read_csv('CountData_HYS1vs2_PD_withHYS_Male.csv')
data.head(1)

S:\PPMI_RNA\Processed\Progression_Analysis\Male


,HudAlphaID,HYS,ENSG00000223972.5,ENSG00000227232.5,ENSG00000278267.1,ENSG00000243485.5,ENSG00000284332.1,ENSG00000237613.2,ENSG00000268020.3,ENSG00000240361.2,...,ENSG00000273739.1,ENSG00000276700.1,ENSG00000276312.1,ENSG00000275757.1,ENSG00000278573.1,ENSG00000276017.1,ENSG00000278817.1,ENSG00000277196.4,ENSG00000278625.1,ENSG00000277374.1
0,5104_SL_0029,1,1,55,2,0,0,0,0,0,...,0,0,3,0,0,0,10,0,0,0


In [10]:
X = data.drop(['HudAlphaID', 'HYS'], axis = 1)
y = data['HYS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   ENSG00000223972.5  ENSG00000227232.5  ENSG00000278267.1  ENSG00000243485.5  \
0                  1                 55                  2                  0   
1                 16                274                 20                  1   

   ENSG00000284332.1  ENSG00000237613.2  ENSG00000268020.3  ENSG00000240361.2  \
0                  0                  0                  0                  0   
1                  0                  0                  0                  1   

   ENSG00000186092.6  ENSG00000238009.6  ...  ENSG00000273739.1  \
0                  0                 85  ...                  0   
1                  0                295  ...                  1   

   ENSG00000276700.1  ENSG00000276312.1  ENSG00000275757.1  ENSG00000278573.1  \
0                  0                  3                  0                  0   
1                 44                  6                  1                  0   

   ENSG00000276017.1  ENSG00000278817.1  ENSG00000277196.4  ENSG00

In [11]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [12]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.42424242424242425

In [13]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100


BorutaPy finished running.

Iteration: 	9 / 100
Confirmed: 	0
Tentative: 	0
Rejected: 	58780


BorutaPy(estimator=RandomForestClassifier(n_estimators=3428,
                                          random_state=RandomState(MT19937) at 0x234B509C440),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x234B509C440, verbose=1)

In [14]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  0


In [15]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [27]:
Top50 = selected_rf_features.iloc[:50, :]
Top50.head(3)

,CpG,Ranking
53566,ENSG00000280433.1,2
5148,ENSG00000236031.1,3
22911,ENSG00000128578.9,4


### Top 50

In [28]:
Top50.to_csv ('HYS1vs2_Boruta_Top50_Male.csv', index = False)
selected_rf_features.to_csv ('HYS1vs2_Boruta_All_Male.csv', index = False)

In [29]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['HYS'] = data['HYS']
df_subset.to_csv ('HYS1vs2_Boruta_Top50_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_8908\1143389863.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['HYS'] = data['HYS']


,ENSG00000259984.1,ENSG00000284138.1,ENSG00000196517.11,ENSG00000142945.12,ENSG00000122477.12,ENSG00000268288.1,ENSG00000234604.2,ENSG00000203721.6,ENSG00000229832.1,ENSG00000236031.1,...,ENSG00000282980.1,ENSG00000277589.1,ENSG00000250751.1,ENSG00000267374.1,ENSG00000124224.16,ENSG00000280433.1,ENSG00000278996.1,ENSG00000212933.1,ENSG00000214322.3,HYS
0,72,14,92,21,31,12,21,332,195,5,...,168,77,6,6,917,190,2,3,0,1
1,81,71,493,52,25,24,13,182,109,9,...,199,68,9,5,1958,56,122,14,3,1


### Top 100

In [30]:
Top100 = selected_rf_features.iloc[:100, :]
Top100.to_csv ('HYS1vs2_Boruta_Top100_Male.csv', index = False)
Top100.head(2)

,CpG,Ranking
53566,ENSG00000280433.1,2
5148,ENSG00000236031.1,3


In [31]:
# create a list of column names to subset from data
col_names = Top100.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['HYS'] = data['HYS']
df_subset.to_csv ('HYS1vs2_Boruta_Top100_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_8908\3681904022.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['HYS'] = data['HYS']


,ENSG00000142583.17,ENSG00000259984.1,ENSG00000228589.3,ENSG00000284138.1,ENSG00000196517.11,ENSG00000142945.12,ENSG00000231613.1,ENSG00000122477.12,ENSG00000268288.1,ENSG00000234604.2,...,ENSG00000275453.2,ENSG00000124224.16,ENSG00000280433.1,ENSG00000278996.1,ENSG00000212933.1,ENSG00000223695.1,ENSG00000214322.3,ENSG00000102003.10,ENSG00000090889.11,HYS
0,82,72,19,14,92,21,8,31,12,21,...,13,917,190,2,3,3,0,19,27,1
1,139,81,35,71,493,52,6,25,24,13,...,4,1958,56,122,14,9,3,50,106,1


### Top 250

In [32]:
Top250 = selected_rf_features.iloc[:250, :]
Top250.to_csv ('HYS1vs2_Boruta_Top250_Male.csv', index = False)
Top250.head(2)

,CpG,Ranking
53566,ENSG00000280433.1,2
5148,ENSG00000236031.1,3


In [33]:
# create a list of column names to subset from data
col_names = Top250.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['HYS'] = data['HYS']
df_subset.to_csv ('HYS1vs2_Boruta_Top250_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_8908\2749445622.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['HYS'] = data['HYS']


,ENSG00000241860.6,ENSG00000049249.8,ENSG00000142583.17,ENSG00000261349.1,ENSG00000259984.1,ENSG00000228589.3,ENSG00000271554.1,ENSG00000116885.18,ENSG00000284138.1,ENSG00000196517.11,...,ENSG00000214322.3,ENSG00000234636.2,ENSG00000102003.10,ENSG00000090889.11,ENSG00000131724.10,ENSG00000232412.1,ENSG00000285018.1,ENSG00000210196.2,ENSG00000277856.1,HYS
0,59,393,82,469,72,19,17,7,14,92,...,0,15,19,27,4711,29,5,44,11,1
1,150,867,139,1054,81,35,35,13,71,493,...,3,27,50,106,5406,33,8,273,26,1


### Top 500

In [34]:
Top500 = selected_rf_features.iloc[:500, :]
Top500.to_csv ('HYS1vs2_Boruta_Top500_Male.csv', index = False)
Top500.head(2)

,CpG,Ranking
53566,ENSG00000280433.1,2
5148,ENSG00000236031.1,3


In [35]:
# create a list of column names to subset from data
col_names = Top500.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['HYS'] = data['HYS']
df_subset.to_csv ('HYS1vs2_Boruta_Top500_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_8908\987567329.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['HYS'] = data['HYS']


,ENSG00000241860.6,ENSG00000222623.1,ENSG00000230021.10,ENSG00000271806.1,ENSG00000049249.8,ENSG00000142583.17,ENSG00000171621.13,ENSG00000162444.11,ENSG00000237276.8,ENSG00000261349.1,...,ENSG00000252627.1,ENSG00000232412.1,ENSG00000221930.6,ENSG00000285018.1,ENSG00000277203.1,ENSG00000211459.2,ENSG00000198763.3,ENSG00000210196.2,ENSG00000277856.1,HYS
0,59,13,27,18,393,82,29,132,11,469,...,26,29,28,5,33,52,59,44,11,1
1,150,36,43,52,867,139,114,204,119,1054,...,18,33,28,8,153,139,503,273,26,1
